In [1]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc, disvg, wsvg
import numpy as np

seg1 = QuadraticBezier(100+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
seg3 = Line(50+100j,400+200j)
#Four Intersections
#seg1 = QuadraticBezier(250+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
#seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
path = Path(seg1, seg2, seg3)

off1 = QuadraticBezier(100+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
off2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
off3 = Line(50+100j,400+200j)

off_path = Path(off1,off2,off3)

In [2]:
import time
def mesure(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"'{func.__name__}': {execution_time:.6f} seconds.")
        return result
    return wrapper

In [147]:
# @mesure
def offset_segment(seg,distance,res=5,prev_end=None) -> list: 
    distance = distance * - 1
    # print()
    if isinstance(seg,Line):
        offset_vector = distance * seg.normal(0)
        if prev_end and (abs(prev_end - (seg.point(0) + offset_vector))<=10):
            offset_seg = Line(prev_end,seg.point(1) + offset_vector)
        else:
            # offset_vector = distance * seg.normal(0)
            offset_seg = Line(seg.point(0) + offset_vector,seg.point(1) + offset_vector)
        return [offset_seg]

    elif isinstance(seg,(Arc, QuadraticBezier)):
        offset_seg = Path()
        # print('prev:', prev_end,'curr:', seg.point(0) + distance * seg.normal(0))
        
        if prev_end and (abs(prev_end - (seg.point(0) + distance * seg.normal(0)))<=10):
            # print(abs(prev_end - (seg.point(0) + distance * seg.normal(0))))
            curr = prev_end
            # print('ocurre',curr)
        else:
            curr = seg.point(0) + distance * seg.normal(0)
            # print('no ocurre')

        # curr = seg.point(0) + distance * seg.normal(0)    
        curr_tan = seg.unit_tangent(0)
        for k in range(1,res+1):
            t=k / res
            next = seg.point(t) + distance * seg.normal(t)
            next_tan = seg.unit_tangent(t)
            
            # Having curr/next as start/end  of each segment,
            # vectors curr_tan/next_tan pointing to a common point
            # can be used as refference for offset segment as bezier curve
            if curr_tan.real == 0:
                X = curr.real
                Y = (next_tan.imag/next_tan.real) * (X - next.real) + next.imag

            elif next_tan.real == 0:
                X = next.real
                Y = (curr_tan.imag/curr_tan.real) * (X - curr.real) + curr.imag

            else:
                Ax = curr.real; Ay = curr.imag
                Bx = next.real; By = next.imag

                ma = curr_tan.imag/curr_tan.real
                mb = next_tan.imag/next_tan.real

                # solving for lines intersection
                X = ((By-Ay) - (mb * Bx - ma * Ax))/(ma-mb)
                Y = (ma*mb * (Bx-Ax) - (By*ma-Ay*mb))/(mb-ma)

            control = complex(X,Y)
            new_element = QuadraticBezier(curr,control,next)

            # Before appending new_element, check if it intersects with some
            # non consecutive previous element, if so, delete all elements between
            # both of them, chop tails and check for continuity.

            offset_seg.append(new_element)
            # print(curr,next)
            curr = next
            curr_tan = next_tan
        
        return list(offset_seg)  #-> Path

    else:
        raise TypeError("Input `segment` should be a Line, "
                        "QuadraticBezier, CubicBezier, or Arc object.")

def offset_path(path: Path,distance,res=5) -> list:
    result_path = Path()
    if path.iscontained: distance = distance * -1
    for i, seg in enumerate(path):
        next_seg = path[(i+1)%len(path)]

        # print('Control:',result_path.end)
        if result_path:
            result_path.extend(offset_segment(seg,distance,res,result_path.end)) ###
        else:
            result_path.extend(offset_segment(seg,distance,res)) ###

        # if seg.normal(1) != next_seg.normal(0):
        if abs(seg.normal(1) - next_seg.normal(0))>0.1:

            result_path.append(Arc(start=seg.point(1) - distance * seg.normal(1), ###
                radius=complex(distance, distance),
                rotation=0,
                large_arc=False,
                sweep=distance<0,
                end=next_seg.point(0) - distance * next_seg.normal(0)))
    return result_path


In [158]:
@mesure
def remove_loops(path:Path) -> Path:
    # window = int(len(path)*0.10)
    nodes = []
    intersections = []
    for i, seg in enumerate(path):
        for j in range(i,len(path)-2):
            other_seg = path[(j+2)%len(path)]
            if intersection := seg.intersect(other_seg):
                if intersection == [(0.0, 1.0)]: continue
                nodes.append(seg.point(intersection[0][0]))
                intersections.append(((i,(j+2)%len(path)),intersection[0]))
                
    #order
    order = [order[0][1] for order in intersections]
    pairs = zip(order,intersections)
    intersections = [x[1] for x in sorted(pairs, key=lambda x: x[0])]
    original_len = len(intersections)
    for elements in intersections[::-1]:
        indexes, t_values = elements[0], elements[1]
        # print(indexes)
        if indexes[1]-indexes[0] == 2:
            path[indexes[0]:indexes[1]+1] = [path[indexes[0]].split(t_values[0])[0],path[indexes[1]].split(t_values[1])[1]]
        else:
            path[indexes[1]:],path[0:indexes[0]+1] = [path[indexes[1]].split(t_values[0])[0]], [path[indexes[0]].split(t_values[0])[1]]

    # print(intersections)
    # print(nodes)
    return nodes

In [168]:
character = char2paths('H')
offset_character = []
path = Path(*character[0][0::])
offset_character.append(
    offset_path(path,-100)
)
# arcs = 4
# lines = 4
# print((len(path)-lines)*5+lines + arcs)
# print(len(offset_character[0]))

arcs = Path()
# print(path)
# print(offset_character[0])
for seg in offset_character[0]:
    if isinstance(seg,Arc):
        arcs.append(seg)


# nodes = [seg.start for seg in offset_character[0][:10] if isinstance(seg,QuadraticBezier)]
# nodes = [seg.start for seg in offset_character[0][:10]]

nodes = remove_loops(offset_character[0])

wsvg(character + [path,offset_character[0]],filename='final.svg',nodes=nodes)
# wsvg(character,nodes=nodes,filename='final.svg')

svgtools.svg
'remove_loops': 0.007048 seconds.


In [13]:
import matplotlib.pyplot as plt
from svgpathtools import Line, QuadraticBezier, CubicBezier, Arc, Path
from svgpathtools import svg2paths, svg2paths2, wsvg
Path.iscontained = False

# @mesure
def char2paths(char): # Implement character with multiple path, maybe ussing parsing

    plt.figure(figsize=(4, 4))
    plt.text(0.5, 0.5, char[0], fontsize=200, ha='center', va='center')
    plt.axis('off')
    filename='.matplot'
    plt.savefig(filename, format='svg')
    plt.close()
    
    paths, _, _ = svg2paths2(filename)
    paths = paths[1].continuous_subpaths()
    filename='svgtools.svg'
    wsvg(paths, filename=filename)
    print(filename)
    return paths #list

# @mesure
def set_contained_attr(paths: list) -> None:
    for path in paths:
        path.iscontained = any(path.is_contained_by(compare) for compare in paths if compare != path)

In [55]:
my_dic = {}
my_dic[(1,1)] = 1
my_dic

{(1, 1): 1}

In [118]:
a = [0,1,2,3,4,5,6]
a[2:4+1] = ['a','b']
a

[0, 1, 'a', 'b', 5, 6]

In [142]:
-2>0

False